In [ ]:
from google.colab import files
import os
from PIL import Image, ImageOps
import numpy as np
import imgaug.augmenters as iaa
import shutil

# Step 1: Upload the dataset
uploaded = files.upload()

# Step 2: Remove existing directory if it exists to avoid conflicts
if os.path.exists('/content/dataset-loz2'):
    shutil.rmtree('/content/dataset-loz2')

# Step 3: Unzip the uploaded dataset (assuming the dataset is a zip file) with overwrite option
!unzip -o dataset-loz2.zip -d dataset-loz2

# Step 4: Define paths for input and output folders
input_folder = '/content/dataset-loz2'
output_folder = '/content/processed_dataset-loz2'

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Augmentation sequence (excluding scaling as per instructions)
augmentation_sequence = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Affine(rotate=(-45, 45)), # rotation
    iaa.AdditiveGaussianNoise(scale=(0.1*255, 0.2*255)), # medium noise
    iaa.ElasticTransformation(alpha=50, sigma=5), # distortions
    iaa.Multiply((0.8, 1.2)), # change brightness
    iaa.LinearContrast((0.75, 1.5)) # change contrast
])

def convert_and_autocontrast(image_path):
    """Convert image to grayscale and apply auto contrast."""
    image = Image.open(image_path).convert('L') # Convert to grayscale
    image = ImageOps.autocontrast(image) # Apply auto contrast
    return image

def cut_image_into_fragments(image, fragment_size=(128, 128)):
    """Cut image into smaller fragments."""
    fragments = []
    image_width, image_height = image.size
    for i in range(0, image_width, fragment_size[0]):
        for j in range(0, image_height, fragment_size[1]):
            box = (i, j, i + fragment_size[0], j + fragment_size[1])
            fragment = image.crop(box)
            fragments.append(fragment)
    return fragments

def augment_and_save_fragments(fragments, output_path, augmentations):
    """Apply augmentations to image fragments and save them."""
    for idx, fragment in enumerate(fragments):
        # Convert to numpy array
        fragment_array = np.array(fragment)

        # Apply augmentations
        images_aug = augmentations(images=[fragment_array])

        # Save augmented images with 5-6 rotations
        for i, img_aug in enumerate(images_aug):
            for rot in range(5, 7): # 5-6 rotations
                img_rot = iaa.Affine(rotate=(rot * 60)).augment_image(img_aug) # rotate each by 60 degrees
                img_rot = Image.fromarray(img_rot)
                fragment_filename = f"fragment_{idx}_aug_{i}_rot_{rot}.jpg"
                img_rot.save(os.path.join(output_path, fragment_filename))

# Process all images in the input folder
for subdir, _, files in os.walk(input_folder):
    for file in files:
        file_path = os.path.join(subdir, file)
        output_subdir = os.path.join(output_folder, os.path.relpath(subdir, input_folder))
        os.makedirs(output_subdir, exist_ok=True)

        # Convert and apply auto contrast
        processed_image = convert_and_autocontrast(file_path)

        # Cut into fragments
        fragments = cut_image_into_fragments(processed_image)

        # Apply augmentations and save
        augment_and_save_fragments(fragments, output_subdir, augmentation_sequence)

print("Image processing and augmentation completed.")

# Create a zip archive of the processed dataset
shutil.make_archive('processed_dataset-loz2', 'zip', '/content/processed_dataset-loz2')

# Ensure correct import of the files module and download the zip file
import google.colab.files as gfiles
gfiles.download('processed_dataset-loz2.zip')

Saving dataset-loz2.zip to dataset-loz2.zip
Archive:  dataset-loz2.zip
   creating: dataset-loz2/dataset2/
   creating: dataset-loz2/dataset2/-20/
   creating: dataset-loz2/dataset2/-20/clear/
  inflating: dataset-loz2/dataset2/-20/clear/25(3-1)пп(20)-1-2.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-1)пп(20)-1-3.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-1)пп(20)-1-4.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-1)пп(20)-1-5.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-2)пп(20)-1-1.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-2)пп(20)-1-2.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-2)пп(20)-1-3.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-2)пп(20)-1-4.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/25(3-2)пп(20)-1-5.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/27(1-1)пп(20)-2-1.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/27(1-1)пп(20)-2-2.jpg  
  inflating: dataset-loz2/dataset2/-20/clear/27(

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Код сверху запускался, файл загружается. Он в последний момент слетел, а заново долго грузить, не стал перезапускать

In [1]:
from google.colab import files

# Upload the zip file
uploaded = files.upload()

# Check the uploaded file
zip_file_path = list(uploaded.keys())[0]
print(f"Uploaded file: {zip_file_path}")


Saving processed_dataset-loz2.zip to processed_dataset-loz2.zip
Uploaded file: processed_dataset-loz2.zip


In [2]:
import os

# List the contents of the current working directory
print(os.listdir('/content'))

# Verify the file path
zip_file_path = '/content/processed_dataset-loz2.zip'
if os.path.exists(zip_file_path):
    print(f"File found: {zip_file_path}")
else:
    print(f"File not found: {zip_file_path}")


['.config', 'processed_dataset-loz2.zip', 'sample_data']
File found: /content/processed_dataset-loz2.zip


In [3]:
import os
import zipfile
import shutil

# Define the extraction folder
extracted_folder_path = '/content/processed_dataset-loz2'

# Check if the uploaded file is a valid zip file
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        print("File is a valid zip file.")

        # Remove existing directory if it exists to avoid conflicts
        if os.path.exists(extracted_folder_path):
            shutil.rmtree(extracted_folder_path)

        # Extract the uploaded zip file
        zip_ref.extractall(extracted_folder_path)

        # List the contents of the extracted folder
        print(os.listdir(extracted_folder_path))
except zipfile.BadZipFile:
    print("Error: The file is not a valid zip file.")


File is a valid zip file.
['dataset2']


In [ ]:
from google.colab import files
import os
import zipfile
import shutil


# Get the uploaded file name
zip_file_path = list(uploaded.keys())[0]
print(f"Uploaded file: {zip_file_path}")

# Step 2: Define the extraction folder
extracted_folder_path = '/content/processed_dataset-loz2'

# Remove existing directory if it exists to avoid conflicts
if os.path.exists(extracted_folder_path):
    shutil.rmtree(extracted_folder_path)

# Check if the uploaded file is a valid zip file and extract it
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        print("File is a valid zip file.")

        # Extract the uploaded zip file
        zip_ref.extractall(extracted_folder_path)

        # List the contents of the extracted folder
        print("Contents of the extracted folder:")
        for root, dirs, files in os.walk(extracted_folder_path):
            for name in dirs:
                print(f"Directory: {os.path.join(root, name)}")
            for name in files:
                print(f"File: {os.path.join(root, name)}")
except zipfile.BadZipFile:
    print("Error: The file is not a valid zip file.")

# Step 3: Function to count the number of images in a directory
def count_images_in_directory(directory, extensions=['.jpg', '.jpeg', '.png']):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                count += 1
    return count

# Count the number of images in the processed_dataset-loz2 directory
num_images = count_images_in_directory(extracted_folder_path)
print(f'Total number of images in processed_dataset-loz2: {num_images}')


Выходные данные были обрезаны до нескольких последних строк (5000).
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_198_aug_0_rot_5.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_26_aug_0_rot_6.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_99_aug_0_rot_5.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_14_aug_0_rot_5.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_133_aug_0_rot_6.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_34_aug_0_rot_5.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_194_aug_0_rot_5.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_141_aug_0_rot_6.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_140_aug_0_rot_6.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_175_aug_0_rot_6.jpg
File: /content/processed_dataset-loz2/dataset2/-25/T/fragment_93_aug_0_rot_6.jpg
File: /content/processed_dataset-lo

In [ ]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define the classes and the nested structure
classes = ['G', 'GP', 'T', 'M', 'clear']
sub_folders = ['-20', '-25', '-30', '-35']

# Initialize lists to hold images and labels
images = []
labels = []

# Load images and labels
for sub_folder in sub_folders:
    for label, class_name in enumerate(classes):
        class_folder = os.path.join(extracted_folder_path, 'dataset2', sub_folder, class_name)
        for filename in os.listdir(class_folder):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(class_folder, filename)
                img = Image.open(img_path).convert('L')  # Convert to grayscale
                img = img.resize((128, 128))  # Resize images to a consistent size
                img_array = np.array(img)
                images.append(img_array)
                labels.append(label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize images
images = images / 255.0

# Reshape images for training
images = np.expand_dims(images, -1)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train_cat = to_categorical(y_train, num_classes=len(classes))
y_test_cat = to_categorical(y_test, num_classes=len(classes))


Define the VAE architecture:

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the encoder
latent_dim = 64

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

encoder_inputs = layers.Input(shape=(128, 128, 1))
x = layers.Conv2D(32, (3, 3), activation='relu', strides=2, padding='same')(encoder_inputs)
x = layers.Conv2D(64, (3, 3), activation='relu', strides=2, padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Sampling()([z_mean, z_log_var])
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')

# Define the decoder
latent_inputs = layers.Input(shape=(latent_dim,))
x = layers.Dense(32 * 32 * 64, activation='relu')(latent_inputs)
x = layers.Reshape((32, 32, 64))(x)
x = layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', strides=2, padding='same')(x)
decoder_outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)
decoder = models.Model(latent_inputs, decoder_outputs, name='decoder')

# Define the VAE
vae_outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, vae_outputs, name='vae')

# Define the VAE loss
reconstruction_loss = tf.keras.losses.binary_crossentropy(encoder_inputs, vae_outputs)
reconstruction_loss *= 128 * 128
kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
kl_loss = tf.reduce_mean(kl_loss) * -0.5
vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')


Train the VAE:

In [ ]:
# Train the VAE
vae.fit(X_train, epochs=20, batch_size=64, validation_data=(X_test, None))

# Save the trained VAE model
vae.save('vae_model.h5')

# Generate synthetic images
num_synthetic_images = 15000
latent_samples = np.random.normal(size=(num_synthetic_images, latent_dim))
synthetic_images = decoder.predict(latent_samples)


Epoch 1/20
120/120 [==============================] - 343s 3s/step - loss: 9656.7275 - val_loss: 9223.7861
Epoch 2/20
120/120 [==============================] - 353s 3s/step - loss: 9154.0889 - val_loss: 9150.2305
Epoch 3/20
120/120 [==============================] - 329s 3s/step - loss: 9110.3848 - val_loss: 9125.9189
Epoch 4/20
120/120 [==============================] - 326s 3s/step - loss: 9091.7178 - val_loss: 9112.3916
Epoch 5/20
120/120 [==============================] - 325s 3s/step - loss: 9077.0273 - val_loss: 9102.0127
Epoch 6/20
120/120 [==============================] - 322s 3s/step - loss: 9064.4795 - val_loss: 9090.6299
Epoch 7/20
120/120 [==============================] - 344s 3s/step - loss: 9053.6943 - val_loss: 9099.3613
Epoch 8/20
120/120 [==============================] - 325s 3s/step - loss: 9044.9268 - val_loss: 9074.0996
Epoch 9/20
120/120 [==============================] - 340s 3s/step - loss: 9034.5244 - val_loss: 9065.4951
Epoch 10/20
120/120 [================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


469/469 [==============================] - 142s 303ms/step


Classify the synthetic images:

In [ ]:
# Prepare labels for synthetic images (randomly for this example)
synthetic_labels = np.random.randint(0, len(classes), num_synthetic_images)

# Define a simple CNN classifier
classifier = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(classes), activation='softmax')
])

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the classifier
classifier.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Save the classifier model
classifier.save('classifier_model.h5')

# Predict the classes of synthetic images
synthetic_images = np.expand_dims(synthetic_images, -1)
predictions = classifier.predict(synthetic_images)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)


Epoch 1/20
120/120 [==============================] - 239s 2s/step - loss: 1.6027 - accuracy: 0.2226 - val_loss: 1.5735 - val_accuracy: 0.2625
Epoch 2/20
120/120 [==============================] - 231s 2s/step - loss: 1.5683 - accuracy: 0.2706 - val_loss: 1.5408 - val_accuracy: 0.2902
Epoch 3/20
120/120 [==============================] - 223s 2s/step - loss: 1.5533 - accuracy: 0.2808 - val_loss: 1.5435 - val_accuracy: 0.2766
Epoch 4/20
120/120 [==============================] - 230s 2s/step - loss: 1.5385 - accuracy: 0.2956 - val_loss: 1.5398 - val_accuracy: 0.2923
Epoch 5/20
120/120 [==============================] - 230s 2s/step - loss: 1.5292 - accuracy: 0.2977 - val_loss: 1.5371 - val_accuracy: 0.2902
Epoch 6/20
120/120 [==============================] - 228s 2s/step - loss: 1.5288 - accuracy: 0.3020 - val_loss: 1.5590 - val_accuracy: 0.2933
Epoch 7/20
120/120 [==============================] - 231s 2s/step - loss: 1.5060 - accuracy: 0.3149 - val_loss: 1.5455 - val_accuracy: 0.2881

In [ ]:
# Save synthesized images
import os
from PIL import Image
import shutil
from google.colab import files

# Define the directory to save the images
output_dir = '/content/synthetic_images_vae'
os.makedirs(output_dir, exist_ok=True)

# Save images as JPEG files
for i, img_array in enumerate(synthetic_images):
    img = Image.fromarray((img_array.squeeze() * 255).astype(np.uint8), 'L')  # Convert array to grayscale image
    img.save(os.path.join(output_dir, f'synthetic_image_{i}.jpg'))

print(f"Synthetic images saved to {output_dir}.")

# Create a ZIP file of the synthetic images
zip_filename = 'synthetic_images_vae.zip'
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', output_dir)

# Download the ZIP file
files.download(f'{zip_filename}')

Synthetic images saved to /content/synthetic_images_vae.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now train the GAN

In [ ]:
import zipfile
import os

# Path to the zip file (assuming it's already uploaded to Colab)
zip_path = '/content/processed_dataset-loz2.zip'
output_dir = '/content/processed_dataset-loz2/'

# Unzipping the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("Dataset unzipped successfully.")

In [ ]:
import numpy as np
import cv2
from glob import glob

def load_images(directory):
    images = []
    for img_path in glob(os.path.join(directory, '**', '*.jpg'), recursive=True):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load images as grayscale
        if img is not None:
            images.append(img)
    images = np.array(images)
    print(f"Loaded {len(images)} images.")
    return images

data_dir = '/content/processed_dataset-loz2/'

# Load images to verify
X_train = load_images(data_dir)

if len(X_train) == 0:
    raise ValueError("No images found. Please check the dataset path and format.")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def build_generator(latent_dim, img_shape):
    model = tf.keras.Sequential([
        layers.Dense(128 * (img_shape[0] // 4) * (img_shape[1] // 4), activation="relu", input_dim=latent_dim),
        layers.Reshape(((img_shape[0] // 4), (img_shape[1] // 4), 128)),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        layers.ReLU(),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(1, kernel_size=3, strides=1, padding="same", activation="tanh"),
        layers.Reshape(img_shape)
    ])
    return model

def build_discriminator(img_shape):
    model = tf.keras.Sequential([
        layers.Conv2D(64, kernel_size=3, strides=2, padding="same", input_shape=img_shape),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Conv2D(256, kernel_size=3, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid"),
    ])
    return model

latent_dim = 100
img_shape = X_train[0].shape + (1,)  # Dynamically set the shape based on the first image

generator = build_generator(latent_dim, img_shape)
discriminator = build_discriminator(img_shape)

discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
discriminator.trainable = False

z = layers.Input(shape=(latent_dim,))
img = generator(z)
validity = discriminator(img)

combined = tf.keras.models.Model(z, validity)
combined.compile(loss="binary_crossentropy", optimizer="adam")


In [ ]:
import matplotlib.pyplot as plt

synthetic_dir = '/content/synthetic_images/'  # Define synthetic_dir globally

def train_gan(generator, discriminator, combined, epochs, batch_size, data_dir, synthetic_dir):
    X_train = load_images(data_dir)
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
    X_train = np.expand_dims(X_train, axis=-1)

    if not os.path.exists(synthetic_dir):
        os.makedirs(synthetic_dir)

    class_names = ['G', 'GP', 'T', 'M', 'clear']
    for class_name in class_names:
        class_dir = os.path.join(synthetic_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.ones((batch_size, 1))

        g_loss = combined.train_on_batch(noise, valid_y)

        print(f"{epoch+1} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

    # Save final images after training
    save_final_images(generator, synthetic_dir, class_names, X_train[0].shape)

def save_final_images(generator, synthetic_dir, class_names, img_shape, latent_dim=100, examples=100):
    noise = np.random.normal(0, 1, (examples * len(class_names), latent_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Normalize to [0, 1]

    for i, img in enumerate(gen_imgs):
        class_name = class_names[i % len(class_names)]
        img_path = os.path.join(synthetic_dir, class_name, f"synthetic_img_{i}.png")
        img_resized = tf.image.resize(img, img_shape[:2]).numpy().squeeze()
        plt.imsave(img_path, img_resized, cmap='gray')  # Save as grayscale

epochs = 500  # Reduced number of epochs for faster training
batch_size = 32

train_gan(generator, discriminator, combined, epochs, batch_size, data_dir, synthetic_dir)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load synthetic images generated by the GAN
def load_synthetic_images(directory):
    datagen = ImageDataGenerator(rescale=1./255)
    synthetic_data = datagen.flow_from_directory(directory, target_size=(64, 64), color_mode='grayscale', batch_size=32, class_mode='categorical')
    return synthetic_data

# Build a CNN classifier
def build_classifier(input_shape, num_classes):
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

input_shape = (64, 64, 1)
num_classes = 5

classifier = build_classifier(input_shape, num_classes)
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

synthetic_data = load_synthetic_images(synthetic_dir)  # Adjust this path accordingly
classifier.fit(synthetic_data, epochs=10)

In [ ]:
from google.colab import files
import shutil

# Create a zip file of the synthetic dataset
shutil.make_archive('/content/synthetic_images_gan', 'zip', synthetic_dir)

# Download the zip file to your local machine
files.download('/content/synthetic_images_gan.zip')

Finally, train the T-CNN

In [5]:
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf

# Data preprocessing
def preprocess_image(img_path):
    img = Image.open(img_path)  # Assuming images are already in grayscale
    img = img.resize((128, 128))  # Resize images to a consistent size
    img_array = np.array(img)
    return img_array

def load_dataset(extracted_folder_path):
    classes = ['G', 'GP', 'T', 'M', 'clear']
    sub_folders = ['-20', '-25', '-30', '-35']
    images = []
    labels = []
    for sub_folder in sub_folders:
        for label, class_name in enumerate(classes):
            class_folder = os.path.join(extracted_folder_path, 'dataset2', sub_folder, class_name)
            for filename in os.listdir(class_folder):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(class_folder, filename)
                    img_array = preprocess_image(img_path)
                    images.append(img_array)
                    labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

# Load and preprocess data
extracted_folder_path = '/content/processed_dataset-loz2'
images, labels = load_dataset(extracted_folder_path)

# Normalize images
images = images / 255.0

# Reshape images for training
images = np.expand_dims(images, -1)

# Convert labels to categorical
labels = to_categorical(labels, num_classes=5)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Create tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1024).batch(64).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64).prefetch(tf.data.experimental.AUTOTUNE)

from tensorflow.keras import layers, models

def build_t_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

input_shape = (128, 128, 1)
num_classes = 5
t_cnn = build_t_cnn(input_shape, num_classes)
t_cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
t_cnn.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate the model
loss, accuracy = t_cnn.evaluate(test_dataset)
print(f'Test accuracy: {accuracy * 100:.2f}%')

# Generate synthetic images based on T-CNN predictions
num_synthetic_images_per_class = 200  # Total of 1000 synthetic images (200 per class)
classes = ['G', 'GP', 'T', 'M', 'clear']
synthetic_images = {class_name: [] for class_name in classes}

for class_idx, class_name in enumerate(classes):
    class_images = X_test[np.argmax(y_test, axis=1) == class_idx]
    for _ in range(num_synthetic_images_per_class):
        idx = np.random.randint(0, class_images.shape[0])
        img = class_images[idx]
        synthetic_images[class_name].append(img)

# Save synthesized images
import shutil
from PIL import Image

output_dir = '/content/synthetic_images_t_cnn'
os.makedirs(output_dir, exist_ok=True)

for class_name, images in synthetic_images.items():
    class_dir = os.path.join(output_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)
    for i, img in enumerate(images):
        img = img.squeeze() * 255
        img = Image.fromarray(img.astype(np.uint8))
        img.save(os.path.join(class_dir, f'synthetic_image_{i}.png'))

# Remove any images in the main folder
for file in os.listdir(output_dir):
    file_path = os.path.join(output_dir, file)
    if os.path.isfile(file_path):
        os.remove(file_path)

# Create a ZIP file of the synthetic images
zip_filename = 'synthetic_images_t_cnn.zip'

shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', output_dir)

# Download the ZIP file
from google.colab import files
files.download(f'{zip_filename}')


Epoch 1/10
120/120 [==============================] - 252s 2s/step - loss: 1.6095 - accuracy: 0.2101 - val_loss: 1.5865 - val_accuracy: 0.2390
Epoch 2/10
120/120 [==============================] - 247s 2s/step - loss: 1.5824 - accuracy: 0.2509 - val_loss: 1.5771 - val_accuracy: 0.2667
Epoch 3/10
120/120 [==============================] - 251s 2s/step - loss: 1.5613 - accuracy: 0.2706 - val_loss: 1.5534 - val_accuracy: 0.2787
Epoch 4/10
120/120 [==============================] - 248s 2s/step - loss: 1.5580 - accuracy: 0.2799 - val_loss: 1.5620 - val_accuracy: 0.2792
Epoch 5/10
120/120 [==============================] - 255s 2s/step - loss: 1.5599 - accuracy: 0.2825 - val_loss: 1.5516 - val_accuracy: 0.2813
Epoch 6/10
120/120 [==============================] - 249s 2s/step - loss: 1.5505 - accuracy: 0.2881 - val_loss: 1.5488 - val_accuracy: 0.2824
Epoch 7/10
120/120 [==============================] - 249s 2s/step - loss: 1.5500 - accuracy: 0.2883 - val_loss: 1.5560 - val_accuracy: 0.2777

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>